<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2*

# Sprint Challenge - Neural Network Foundations

Table of Problems

1. [Defining Neural Networks](#Q1)
2. [Perceptron on XOR Gates](#Q2)
3. [Multilayer Perceptron](#Q3)
4. [Keras MMP](#Q4)

<a id="Q1"></a>
## 1. Define the following terms:

- **Neuron:** An individual node in a layer of a Neural Network. Each neuron sends the weighted sum of its inputs through an activation function. The output of a neuron is the value of the activation function.

- **Input Layer:** The first layer of a Neural Network which takes the values of features as inputs.

- **Hidden Layer:** Any layer between the input and output layer. These layers add complexity.

- **Output Layer:** The final layer of a Neural Network which returns the output we're looking for.

- **Activation:** The function that determines the final output of a Neuron. This is analogous to "how much a Neuron fires" in the brain.

- **Backpropagation:** The process by which weights in the Neural Network are adjusted.


## 2. Perceptron on XOR Gates <a id="Q3=2"></a>

Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

|x1	|x2|x3|	y|
|---|---|---|---|
1|	1|	1|	1|
1|	0|	1|	0|
0|	1|	1|	0|
0|	0|	1|	0|

In [1]:
import numpy as np

np.random.seed(42)

# Set the input and output
inputs = np.array([
    [1, 1, 1],
    [1, 0, 1],
    [0, 1, 1],
    [0, 0, 1]
])

correct_outputs = [[1], [0], [0], [0]]

In [3]:
# Define the activation function & derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid
def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

In [4]:
# Keep weights bounded [-1, 1]
weights = 2 * np.random.random((3, 1)) - 1
weights

array([[-0.25091976],
       [ 0.90142861],
       [ 0.46398788]])

In [5]:
# trian for 10,000 epochs
for iteration in range(10000):
    # weighted sum
    weighted_sum = np.dot(inputs, weights)
    
    # Activate using Sigmoid
    activated_output = sigmoid(weighted_sum)
    
    # Errors & Adjustments
    error = correct_outputs - activated_output
    adjustments = error * sigmoid_derivative(activated_output)
    
    # Update weights
    weights += np.dot(inputs.T, adjustments)
    
print('Output after Training:')
print(activated_output)

Output after Training:
[[9.96429406e-01]
 [2.00908516e-03]
 [2.00908516e-03]
 [1.45223694e-08]]


## 3. Multilayer Perceptron <a id="Q3"></a>

Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights.
Your network must have one hidden layer.
You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
Train your model on the Heart Disease dataset from UCI:



In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df = df.sample(frac=1)
print(df.shape)
df.head()

(303, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
179,57,1,0,150,276,0,0,112,1,0.6,1,1,1,0
228,59,1,3,170,288,0,0,159,0,0.2,1,0,3,0
111,57,1,2,150,126,1,1,173,0,0.2,2,1,3,1
246,56,0,0,134,409,0,0,150,1,1.9,1,2,3,0
60,71,0,2,110,265,1,0,130,0,0.0,2,1,2,1


In [3]:
# Select Features & train_test split
features = df.columns.tolist()[0:13]
target = df.columns.tolist()[13]

# Split by features & target
X = df[features]
y = df[target]

# Instantiate scaler
scaler = StandardScaler()
# Normalize X
X = pd.DataFrame(scaler.fit_transform(X), columns = features)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [4]:
print(X_train.shape)
X_test.shape

(242, 13)


(61, 13)

In [54]:
class MLP:
    def __init__(self):
        # Set up Architecture
        self.inputs = 13
        self.hiddenNodes = 4
        self.output = 1
        
        # Initialize Weights
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
        self.weights2 = np.random.rand(self.hiddenNodes, self.output)
        
    # Define the activation function & derivative
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # Derivative of sigmoid
    def sigmoid_derivative(self, x):
        sx = self.sigmoid(x)
        return sx * (1-sx)
    
    # Forward Propogation
    def feed_forward(self, X):
        # Activation of Weighted Sum
        self.hidden_sum = np.dot(X, self.weights1)
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Next Layer
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    # Backward Propagation
    def feed_backward(self, X, y, y_pred):
        # Output --> Hidden
        self.output_error = y.values - y_pred
        self.output_delta = self.output_error.dot(self.sigmoid_derivative(y_pred))
        
        # Hidden --> Input
        self.input_error = self.output_delta.dot(self.weights2.T)
        self.input_delta = self.input_error * self.sigmoid_derivative(self.activated_hidden)
        
        # Update Weights
        self.weights1 += X.T.dot(self.input_delta)
        self.weights2 += self.activated_hidden.T.dot(self.output_delta)
        
    # Train MLP
    def train(self, X, y):
        y_pred = self.feed_forward(X)
        self.feed_backward(X, y, y_pred)

In [55]:
# Instantiate MLP
mlp = MLP()

In [61]:
# Set number of Epochs
for i in range(100000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 10000 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print("Loss: \n", str(np.mean(np.square(y.values - mlp.feed_forward(X)))))
    mlp.train(X,y)

+---------EPOCH 1---------+
Loss: 
 0.45544554455445546
+---------EPOCH 2---------+
Loss: 
 0.5445544554455446
+---------EPOCH 3---------+
Loss: 
 0.45544554455445546
+---------EPOCH 4---------+
Loss: 
 0.45544554455445546
+---------EPOCH 5---------+
Loss: 
 0.5445544554455446


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


+---------EPOCH 10000---------+
Loss: 
 0.45544554455445546
+---------EPOCH 20000---------+
Loss: 
 0.45544554455445546
+---------EPOCH 30000---------+
Loss: 
 0.45544554455445546
+---------EPOCH 40000---------+
Loss: 
 0.5445544554455446
+---------EPOCH 50000---------+
Loss: 
 0.5445544554455446
+---------EPOCH 60000---------+
Loss: 
 0.5445544554455446


KeyboardInterrupt: 

## 4. Keras MMP <a id="Q4"></a>

Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy.
Use the Heart Disease Dataset (binary classification)
Use an appropriate loss function for a binary classification task
Use an appropriate activation function on the final layer of your network.
Train your model using verbose output for ease of grading.
Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
When hyperparameter tuning, show you work by adding code cells for each new experiment.
Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [47]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df = df.sample(frac=1)
print(df.shape)
df.head()

(303, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
138,57,1,0,110,201,0,1,126,1,1.5,1,0,1,1
27,51,1,2,110,175,0,1,123,0,0.6,2,0,2,1
49,53,0,0,138,234,0,0,160,0,0.0,2,0,2,1
154,39,0,2,138,220,0,1,152,0,0.0,1,0,2,1
141,43,1,0,115,303,0,1,181,0,1.2,1,0,2,1


In [48]:
# Select Features & train_test split
features = df.columns.tolist()[0:13]
target = df.columns.tolist()[13]

# Split by features & target
X = df[features]
y = df[target]

# Instantiate scaler
scaler = StandardScaler()
# Normalize X
X = pd.DataFrame(scaler.fit_transform(X), columns = features)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


### Keras Model

In [63]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV

import numpy as np

np.random.seed(812)

In [53]:
# Create model function for Keras Classifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=13, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [61]:
# Check baseline model stats
model = create_model()

# Fit
model.fit(X_train, y_train, epochs=10)

# Evaluate
model.evaluate(X_test, y_test)

Epoch 1/10
242/242 [==============================] - 0s 517us/sample - loss: 0.7311 - acc: 0.5207
Epoch 2/10
242/242 [==============================] - 0s 93us/sample - loss: 0.7029 - acc: 0.5455
Epoch 3/10
242/242 [==============================] - 0s 125us/sample - loss: 0.6763 - acc: 0.5785
Epoch 4/10
242/242 [==============================] - 0s 125us/sample - loss: 0.6536 - acc: 0.6116
Epoch 5/10
242/242 [==============================] - 0s 167us/sample - loss: 0.6314 - acc: 0.6405
Epoch 6/10
242/242 [==============================] - 0s 84us/sample - loss: 0.6103 - acc: 0.6694
Epoch 7/10
242/242 [==============================] - 0s 56us/sample - loss: 0.5914 - acc: 0.6818
Epoch 8/10
242/242 [==============================] - 0s 178us/sample - loss: 0.5740 - acc: 0.6983
Epoch 9/10
242/242 [==============================] - 0s 61us/sample - loss: 0.5567 - acc: 0.7066
Epoch 10/10
61/61 [==============================] - 0s 610us/sample - loss: 0.5605 - acc: 0.6721


[0.560492181387104, 0.6721311]

**Baseline Accuracy: 67.21%**

### Tune Batch Size

In [58]:
# Define search params
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [10]}

In [65]:
# New model
model = KerasClassifier(build_fn=create_model, verbose=1)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X, y)

Epoch 1/10
202/202 [==============================] - 0s 883us/sample - loss: 0.7092 - acc: 0.5891
Epoch 2/10
202/202 [==============================] - 0s 254us/sample - loss: 0.6597 - acc: 0.6436
Epoch 3/10
202/202 [==============================] - 0s 138us/sample - loss: 0.6157 - acc: 0.6832
Epoch 4/10
202/202 [==============================] - 0s 261us/sample - loss: 0.5806 - acc: 0.7079
Epoch 5/10
202/202 [==============================] - 0s 253us/sample - loss: 0.5486 - acc: 0.7376
Epoch 6/10
202/202 [==============================] - 0s 330us/sample - loss: 0.5221 - acc: 0.7772
Epoch 7/10
202/202 [==============================] - 0s 257us/sample - loss: 0.4990 - acc: 0.7970
Epoch 8/10
202/202 [==============================] - 0s 280us/sample - loss: 0.4786 - acc: 0.8069
Epoch 9/10
202/202 [==============================] - 0s 217us/sample - loss: 0.4601 - acc: 0.8168
Epoch 10/10
202/202 [==============================] - 0s 182us/sample - loss: 0.4379 - acc: 0.8317
Epoch 1/1

202/202 [==============================] - 0s 661us/sample - loss: 1.1438 - acc: 0.4356
Epoch 2/10
202/202 [==============================] - 0s 52us/sample - loss: 1.0950 - acc: 0.4406
Epoch 3/10
202/202 [==============================] - 0s 73us/sample - loss: 1.0548 - acc: 0.4455
Epoch 4/10
202/202 [==============================] - 0s 109us/sample - loss: 1.0175 - acc: 0.4554
Epoch 5/10
202/202 [==============================] - 0s 120us/sample - loss: 0.9853 - acc: 0.4703
Epoch 6/10
202/202 [==============================] - 0s 100us/sample - loss: 0.9528 - acc: 0.4752
Epoch 7/10
202/202 [==============================] - 0s 108us/sample - loss: 0.9192 - acc: 0.4802
Epoch 8/10
202/202 [==============================] - 0s 93us/sample - loss: 0.8912 - acc: 0.4851
Epoch 9/10
202/202 [==============================] - 0s 117us/sample - loss: 0.8647 - acc: 0.5000
Epoch 10/10
202/202 [==============================] - 0s 180us/sample - loss: 0.8253 - acc: 0.5099
Epoch 1/10
202/202 [===

Epoch 2/10
202/202 [==============================] - 0s 33us/sample - loss: 0.9338 - acc: 0.4505
Epoch 3/10
202/202 [==============================] - 0s 30us/sample - loss: 0.9123 - acc: 0.4554
Epoch 4/10
202/202 [==============================] - 0s 34us/sample - loss: 0.8907 - acc: 0.4554
Epoch 5/10
202/202 [==============================] - 0s 72us/sample - loss: 0.8716 - acc: 0.4653
Epoch 6/10
202/202 [==============================] - 0s 46us/sample - loss: 0.8522 - acc: 0.4752
Epoch 7/10
202/202 [==============================] - 0s 70us/sample - loss: 0.8327 - acc: 0.4950
Epoch 8/10
202/202 [==============================] - 0s 103us/sample - loss: 0.8144 - acc: 0.5000
Epoch 9/10
202/202 [==============================] - 0s 122us/sample - loss: 0.7971 - acc: 0.5050
Epoch 10/10
202/202 [==============================] - 0s 28us/sample - loss: 0.7688 - acc: 0.5099
Epoch 1/10
202/202 [==============================] - 0s 834us/sample - loss: 0.6935 - acc: 0.5594
Epoch 2/10
202/2

In [66]:
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.8052805264790853 using {'batch_size': 10, 'epochs': 10}


### Tune Epochs

In [67]:
# Define search params
param_grid = {'batch_size': [10],
              'epochs': [5, 10, 20, 50, 100]}

In [68]:
# New model
model = KerasClassifier(build_fn=create_model, verbose=1)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X, y)

Epoch 1/5
202/202 [==============================] - 0s 1ms/sample - loss: 0.6224 - acc: 0.7228
Epoch 2/5
202/202 [==============================] - 0s 280us/sample - loss: 0.5771 - acc: 0.7426
Epoch 3/5
202/202 [==============================] - 0s 222us/sample - loss: 0.5418 - acc: 0.7723
Epoch 4/5
202/202 [==============================] - 0s 223us/sample - loss: 0.5089 - acc: 0.8020
Epoch 5/5
202/202 [==============================] - 0s 191us/sample - loss: 0.4693 - acc: 0.8317
Epoch 1/5
202/202 [==============================] - 0s 1ms/sample - loss: 0.8373 - acc: 0.3366
Epoch 2/5
202/202 [==============================] - 0s 424us/sample - loss: 0.7852 - acc: 0.3911
Epoch 3/5
202/202 [==============================] - 0s 549us/sample - loss: 0.7411 - acc: 0.4505
Epoch 4/5
202/202 [==============================] - 0s 469us/sample - loss: 0.7014 - acc: 0.5198
Epoch 5/5
202/202 [==============================] - 0s 110us/sample - loss: 0.6496 - acc: 0.6337
Epoch 1/5
202/202 [=====

202/202 [==============================] - 0s 211us/sample - loss: 0.6630 - acc: 0.5891
Epoch 7/20
202/202 [==============================] - 0s 224us/sample - loss: 0.6232 - acc: 0.6535
Epoch 8/20
202/202 [==============================] - 0s 158us/sample - loss: 0.5880 - acc: 0.6931
Epoch 9/20
202/202 [==============================] - 0s 217us/sample - loss: 0.5583 - acc: 0.7426
Epoch 10/20
202/202 [==============================] - 0s 182us/sample - loss: 0.5288 - acc: 0.7574
Epoch 11/20
202/202 [==============================] - 0s 245us/sample - loss: 0.5034 - acc: 0.7822
Epoch 12/20
202/202 [==============================] - 0s 161us/sample - loss: 0.4803 - acc: 0.7970
Epoch 13/20
202/202 [==============================] - 0s 242us/sample - loss: 0.4618 - acc: 0.8119
Epoch 14/20
202/202 [==============================] - 0s 189us/sample - loss: 0.4449 - acc: 0.8168
Epoch 15/20
202/202 [==============================] - 0s 258us/sample - loss: 0.4310 - acc: 0.8267
Epoch 16/20
202

202/202 [==============================] - 0s 245us/sample - loss: 0.2865 - acc: 0.8812
Epoch 45/50
202/202 [==============================] - 0s 252us/sample - loss: 0.2855 - acc: 0.8812
Epoch 46/50
202/202 [==============================] - 0s 192us/sample - loss: 0.2837 - acc: 0.8812
Epoch 47/50
202/202 [==============================] - 0s 253us/sample - loss: 0.2822 - acc: 0.8812
Epoch 48/50
202/202 [==============================] - 0s 185us/sample - loss: 0.2810 - acc: 0.8812
Epoch 49/50
202/202 [==============================] - 0s 255us/sample - loss: 0.2794 - acc: 0.8812
Epoch 50/50
202/202 [==============================] - 0s 118us/sample - loss: 0.2761 - acc: 0.8861
Epoch 1/50
202/202 [==============================] - 0s 1ms/sample - loss: 0.9217 - acc: 0.4901
Epoch 2/50
202/202 [==============================] - 0s 244us/sample - loss: 0.8410 - acc: 0.5446
Epoch 3/50
202/202 [==============================] - 0s 203us/sample - loss: 0.7761 - acc: 0.5792
Epoch 4/50
202/20

202/202 [==============================] - 0s 245us/sample - loss: 0.3501 - acc: 0.8713
Epoch 23/50
202/202 [==============================] - 0s 237us/sample - loss: 0.3466 - acc: 0.8713
Epoch 24/50
202/202 [==============================] - 0s 230us/sample - loss: 0.3434 - acc: 0.8713
Epoch 25/50
202/202 [==============================] - 0s 242us/sample - loss: 0.3399 - acc: 0.8713
Epoch 26/50
202/202 [==============================] - 0s 162us/sample - loss: 0.3371 - acc: 0.8713
Epoch 27/50
202/202 [==============================] - 0s 224us/sample - loss: 0.3339 - acc: 0.8713
Epoch 28/50
202/202 [==============================] - 0s 188us/sample - loss: 0.3310 - acc: 0.8713
Epoch 29/50
202/202 [==============================] - 0s 250us/sample - loss: 0.3279 - acc: 0.8762
Epoch 30/50
202/202 [==============================] - 0s 157us/sample - loss: 0.3255 - acc: 0.8762
Epoch 31/50
202/202 [==============================] - 0s 232us/sample - loss: 0.3238 - acc: 0.8812
Epoch 32/50


202/202 [==============================] - 0s 264us/sample - loss: 0.2660 - acc: 0.9010
Epoch 52/100
202/202 [==============================] - 0s 277us/sample - loss: 0.2639 - acc: 0.9059
Epoch 53/100
202/202 [==============================] - 0s 375us/sample - loss: 0.2621 - acc: 0.9010
Epoch 54/100
202/202 [==============================] - 0s 315us/sample - loss: 0.2607 - acc: 0.9010
Epoch 55/100
202/202 [==============================] - 0s 325us/sample - loss: 0.2590 - acc: 0.9010
Epoch 56/100
202/202 [==============================] - 0s 326us/sample - loss: 0.2576 - acc: 0.9010
Epoch 57/100
202/202 [==============================] - 0s 363us/sample - loss: 0.2561 - acc: 0.9059
Epoch 58/100
202/202 [==============================] - 0s 318us/sample - loss: 0.2556 - acc: 0.9010
Epoch 59/100
202/202 [==============================] - 0s 250us/sample - loss: 0.2542 - acc: 0.8960
Epoch 60/100
202/202 [==============================] - 0s 202us/sample - loss: 0.2526 - acc: 0.8960
Epo

202/202 [==============================] - 0s 264us/sample - loss: 0.3510 - acc: 0.8465
Epoch 31/100
202/202 [==============================] - 0s 223us/sample - loss: 0.3475 - acc: 0.8416
Epoch 32/100
202/202 [==============================] - 0s 202us/sample - loss: 0.3454 - acc: 0.8416
Epoch 33/100
202/202 [==============================] - 0s 253us/sample - loss: 0.3423 - acc: 0.8416
Epoch 34/100
202/202 [==============================] - 0s 199us/sample - loss: 0.3400 - acc: 0.8515
Epoch 35/100
202/202 [==============================] - 0s 256us/sample - loss: 0.3370 - acc: 0.8515
Epoch 36/100
202/202 [==============================] - 0s 201us/sample - loss: 0.3348 - acc: 0.8515
Epoch 37/100
202/202 [==============================] - 0s 260us/sample - loss: 0.3327 - acc: 0.8515
Epoch 38/100
202/202 [==============================] - 0s 200us/sample - loss: 0.3296 - acc: 0.8564
Epoch 39/100
202/202 [==============================] - 0s 264us/sample - loss: 0.3275 - acc: 0.8564
Epo

202/202 [==============================] - 0s 281us/sample - loss: 0.5682 - acc: 0.7129
Epoch 10/100
202/202 [==============================] - 0s 221us/sample - loss: 0.5458 - acc: 0.7376
Epoch 11/100
202/202 [==============================] - 0s 201us/sample - loss: 0.5252 - acc: 0.7525
Epoch 12/100
202/202 [==============================] - 0s 259us/sample - loss: 0.5056 - acc: 0.7772
Epoch 13/100
202/202 [==============================] - 0s 200us/sample - loss: 0.4900 - acc: 0.7871
Epoch 14/100
202/202 [==============================] - 0s 240us/sample - loss: 0.4752 - acc: 0.7970
Epoch 15/100
202/202 [==============================] - 0s 251us/sample - loss: 0.4623 - acc: 0.8069
Epoch 16/100
202/202 [==============================] - 0s 190us/sample - loss: 0.4509 - acc: 0.8119
Epoch 17/100
202/202 [==============================] - 0s 252us/sample - loss: 0.4410 - acc: 0.8069
Epoch 18/100
202/202 [==============================] - 0s 215us/sample - loss: 0.4313 - acc: 0.8119
Epo

202/202 [==============================] - 0s 565us/sample - loss: 0.2645 - acc: 0.8911
Epoch 91/100
202/202 [==============================] - 0s 234us/sample - loss: 0.2638 - acc: 0.8960
Epoch 92/100
202/202 [==============================] - 0s 211us/sample - loss: 0.2626 - acc: 0.8911
Epoch 93/100
202/202 [==============================] - 0s 229us/sample - loss: 0.2622 - acc: 0.8911
Epoch 94/100
202/202 [==============================] - 0s 169us/sample - loss: 0.2608 - acc: 0.8911
Epoch 95/100
202/202 [==============================] - 0s 307us/sample - loss: 0.2601 - acc: 0.8911
Epoch 96/100
202/202 [==============================] - 0s 270us/sample - loss: 0.2595 - acc: 0.8960
Epoch 97/100
202/202 [==============================] - 0s 331us/sample - loss: 0.2587 - acc: 0.8960
Epoch 98/100
202/202 [==============================] - 0s 311us/sample - loss: 0.2571 - acc: 0.8960
Epoch 99/100
202/202 [==============================] - 0s 355us/sample - loss: 0.2566 - acc: 0.8960
Epo

In [69]:
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.8283828298250834 using {'batch_size': 10, 'epochs': 20}


### Best Model

In [101]:
# New model
model = create_model()
model.fit(X_train, y_train, batch_size=10, epochs=20)

Epoch 1/20
242/242 [==============================] - 0s 1ms/sample - loss: 0.6473 - acc: 0.6446
Epoch 2/20
242/242 [==============================] - 0s 388us/sample - loss: 0.5906 - acc: 0.7066
Epoch 3/20
242/242 [==============================] - 0s 320us/sample - loss: 0.5466 - acc: 0.7438
Epoch 4/20
242/242 [==============================] - 0s 462us/sample - loss: 0.5088 - acc: 0.7727
Epoch 5/20
242/242 [==============================] - 0s 286us/sample - loss: 0.4775 - acc: 0.7893
Epoch 6/20
242/242 [==============================] - 0s 421us/sample - loss: 0.4519 - acc: 0.8099
Epoch 7/20
242/242 [==============================] - 0s 258us/sample - loss: 0.4298 - acc: 0.8099
Epoch 8/20
242/242 [==============================] - 0s 330us/sample - loss: 0.4108 - acc: 0.8223
Epoch 9/20
242/242 [==============================] - 0s 412us/sample - loss: 0.3946 - acc: 0.8388
Epoch 10/20
242/242 [==============================] - 0s 1ms/sample - loss: 0.3815 - acc: 0.8388
Epoch 11/20
2

In [102]:
# Evaluate model
model.evaluate(X_test, y_test)

61/61 [==============================] - 0s 3ms/sample - loss: 0.5341 - acc: 0.7377


[0.5341193627138607, 0.73770493]

**Final Accuracy: 73.77%**